In [14]:
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from transformers import AutoTokenizer, AutoModel
from typing import List, Optional, Any

In [15]:
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS  # 向量数据库

In [16]:
# coding:utf-8
# 导入必备的工具包
from langchain.prompts import PromptTemplate
#from get_vector import *
#from model import ChatGLM2
# 加载FAISS向量库
EMBEDDING_MODEL = '/mnt/workspace/logistics/m3e-base'
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
db = FAISS.load_local('/mnt/workspace/logistics/faiss/camp', embeddings,allow_dangerous_deserialization=True)

In [17]:
# 自定义GLM类
class ChatGLM2(LLM):
    max_token: int = 4096
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history = []

    def __init__(self):
        super().__init__()

    @property
    def _llm_type(self) -> str:
        return "custom_chatglm2"

    # 定义load_model的方法
    def load_model(self, model_path=None):
        # 加载分词器
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        # 加载模型
        self.model = AutoModel.from_pretrained(model_path, trust_remote_code=True).float()
        #gpu
        #self.model = AutoModel.from_pretrained(model_path, trust_remote_code=True).cuda()

    # 定义_call方法：进行模型的推理
    def _call(self,prompt: str, stop: Optional[List[str]] = None) -> str:
        response, _ = self.model.chat(self.tokenizer,
                                        prompt,
                                        history=self.history,
                                        temperature=self.temperature,
                                        top_p=self.top_p)

        if stop is not None:
            response = enforce_stop_tokens(response, stop)

        self.history = self.history + [[None, response]]
        return response

In [18]:
def get_related_content(related_docs):
    related_content = []
    for doc in related_docs:
        related_content.append(doc.page_content.replace('\n\n', '\n'))
    return '\n'.join(related_content)

In [19]:
def define_prompt(question):
    #question = '我买的商品来自于哪个仓库，从哪出发的，预计什么到达'
    docs = db.similarity_search('', k=1)
    # print(f'docs-->{docs}')
    related_docs = get_related_content(docs)

    # 构建模板
    PROMPT_TEMPLATE = """
           基于以下已知信息，简洁和专业的来回答用户的问题。不允许在答案中添加编造成分。
           已知内容:
           {context}
           问题:
           {question}"""
    prompt = PromptTemplate(input_variables=["context", "question"],
                            template=PROMPT_TEMPLATE)

    my_prompt = prompt.format(context=related_docs,
                                question=question)
    return my_prompt

In [20]:
def qa(question):
    llm = ChatGLM2()
    llm.load_model('/mnt/workspace/logistics/chatglm2-6b')
    my_prompt = define_prompt(question)
    result = llm(my_prompt)
    return result

In [22]:
if __name__ == '__main__':
    question = '我买的商品来自于哪个仓库，从哪出发的，预计什么到达'
    result = qa(question)
    print(f'result-->{result}')

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.24s/it]


result-->尊敬的用户，根据您提供的信息，我已为您查询到商品存储于深圳市东方仓储中心。商品为电子产品，且预计运输时间为3天。请您前往该仓库领取您的商品。如有疑问，请随时向我提问。


In [23]:
result = qa('我买的商品运输方式是什么')
print(f'result-->{result}')

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.20s/it]


result-->根据提供的信息,商品的运输方式是陆运。具体的运输路线是从广州出发,前往重庆,预计运输时间为3天。商品目前存放在深圳市的东方仓储中心,存储条件为常温仓储,当前库存量为1000件。


In [24]:
result = qa('我买的商品物流编号是什么')
print(f'result-->{result}')

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.23s/it]


result-->根据已知信息，我无法提供商品的物流编号。建议您在购买商品后，通过订单或商品包装上的物流信息查询工具查询物流编号。


In [25]:
result = qa('我买的产品的物流公司是那个')
print(f'result-->{result}')

Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.21s/it]


result-->根据提供的信息，我无法确定你购买的产品由哪个物流公司运输。因为缺少有关物流公司的详细信息，如公司名称、联系方式等。建议您提供更多关于物流公司的信息，以便我更好地帮助您解决问题。
